In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:

import numpy as np
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/dataset/train.csv')
data=data.loc[:,['Gender', 'Married', 'Education','Self_Employed', 'Credit_History']].fillna('NA')
class Root:
  def __init__(self,val=None):
    self.val=val
    self.child=[]
    self.feature_cat=[]
    self.flag=0
class DecisionTreeClassifer(Root):
  def distribution(self,x):
    dic={}
    for element in x:
      if element not in dic.keys():
        dic[element]=1
      else:
        dic[element]+=1
      return dic.values()

  '''def gini(x):
    p = x.value_counts()/x.shape[0]
    gini_val = 1-np.sum(p**2)
    return(gini_val)'''

  def entropy(self,x):
    n=x.shape[0]
    x=np.array(list(self.distribution(x)))/n
    #return np.multiply(-np.log2(x),x)
    return np.sum(-np.log2(x)*x)
  def entropy_feature(self,x):
    ent=0
    for element in x.iloc[:,0].unique():
      t=x.loc[x.iloc[:,0]==element,x.columns[-1]]
      ent+=(self.entropy(t)*t.shape[0]/x.shape[0])
    return ent
  def tree(self,data,root):
    if data.shape[1]==1:
      maximum=0
      feature=None
      for cat in data.iloc[:,0].unique():
        t=data.loc[data.iloc[:,0]==cat,:].shape[0]
        if maximum<t:
          maximum=t
          feature=cat
      root.val=feature
      root.flag=1
      return
    ent_list=[]
    for i in range(data.shape[1]-1):
      ent_list.append(self.entropy_feature(data.iloc[:,[i,-1]]))
    ent_list=np.array(ent_list)
    f=ent_list.argmin()
    root.val=data.columns[f]
    for cat in data.iloc[:,f].unique():
      new_data=data.loc[data.iloc[:,f]==cat,list(data.columns[:f])+list(data.columns[f+1:])]
      new_child=Root()
      root.child.append(new_child)
      root.feature_cat.append(cat)
      self.tree(new_data,new_child)
  def print_tree(self):
    decision_tree=self.root
    queue=[decision_tree]
    while len(queue)!=0:
      next=[]
      current=[]
      while len(queue)!=0:
        current.append(queue.pop(0))
        next.extend(current[-1].child)
      queue.extend(next)
      for element in current:
        if element.flag==0:
          for e in element.feature_cat:
            print('|',str(element.val),'=',e,end='|\t\t')
        else:
          print('|',element.val,'|',end='\t')
      print('\n')
  def fit(self,data):
    self.root=Root()
    self.tree(data,self.root)
  def test(self,data):
    root=self.root
    t=0
    for row in range(data.shape[0]):
      while 1:
        if root.flag==0:
          i=0
          while root.feature_cat[i]!=data.loc[row,root.val]:
            i+=1
          root=root.child[i]
        else:
          #print(root.val)
          if root.val==data.iloc[row,-1]:
            t+=1
          break
    print('Accuracy = ',100*t/data.shape[0])
tree=DecisionTreeClassifer()
tree.fit(data)
tree.print_tree()
test_data=pd.read_csv('/content/drive/MyDrive/dataset/test.csv')
test_data=test_data.loc[:,['Gender', 'Married', 'Education','Self_Employed', 'Credit_History']].fillna('NA')
tree.test(test_data)

| Education = Graduate|		| Education = Not Graduate|		

| Married = No|		| Married = Yes|		| Married = NA|		| Gender = Male|		| Gender = NA|		| Gender = Female|		

| Gender = Male|		| Gender = Female|		| Gender = NA|		| Gender = Male|		| Gender = Female|		| Gender = NA|		| Gender = Male|		| Gender = Female|		| Married = Yes|		| Married = No|		| Married = Yes|		| Married = Yes|		| Married = No|		

| Self_Employed = No|		| Self_Employed = Yes|		| Self_Employed = NA|		| Self_Employed = No|		| Self_Employed = NA|		| Self_Employed = Yes|		| Self_Employed = No|		| Self_Employed = Yes|		| Self_Employed = No|		| Self_Employed = Yes|		| Self_Employed = NA|		| Self_Employed = No|		| Self_Employed = Yes|		| Self_Employed = NA|		| Self_Employed = No|		| Self_Employed = Yes|		| Self_Employed = No|		| Self_Employed = No|		| Self_Employed = No|		| Self_Employed = Yes|		| Self_Employed = NA|		| Self_Employed = No|		| Self_Employed = NA|		| Self_Employed = Yes|		| Self_Employed = No|		| Self_Employed =